In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
pwd


In [ ]:
%cd gdrive/MyDrive/

In [ ]:
character_segmentation

In [ ]:
%cd Ancient-Tamil-Script-Recognition-master

In [ ]:
#Importing necessary libraries
import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy.ndimage import interpolation as inter

In [ ]:
def correct_skew(image, delta=1, limit=5):
    def determine_score(arr, angle):
        data = inter.rotate(arr, angle, reshape=False, order=0)
        histogram = np.sum(data, axis=1)
        score = np.sum((histogram[1:] - histogram[:-1]) ** 2)
        return histogram, score

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1] 

    scores = []
    angles = np.arange(-limit, limit + delta, delta)
    for angle in angles:
        histogram, score = determine_score(thresh, angle)
        scores.append(score)

    best_angle = angles[scores.index(max(scores))]

    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, best_angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, \
              borderMode=cv2.BORDER_REPLICATE)

    return best_angle, rotated

In [ ]:
#reading the original image using OpenCV
image = cv2.imread('TEST_PALM_1.jpg')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Converting the colored image to greyscale
angle, rotated = correct_skew(image)
print(angle)
cv2.imwrite('rotated.jpg', rotated)
gray = cv2.cvtColor(rotated,cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]


In [ ]:
# Remove horizontal lines
horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40,1))
remove_horizontal = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
cnts = cv2.findContours(remove_horizontal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    cv2.drawContours(rotated, [c], -1, (255,255,255), 5)

In [ ]:
# Remove vertical lines
vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,40))
remove_vertical = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, vertical_kernel, iterations=2)
cnts = cv2.findContours(remove_vertical, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    cv2.drawContours(rotated, [c], -1, (255,255,255), 5)

In [ ]:
gray = cv2.cvtColor(rotated,cv2.COLOR_BGR2GRAY)

In [ ]:
#applying median filter for Salt and pepper/impulse noise
filter1 = cv2.medianBlur(gray,5)
cv2_imshow(filter1)

In [ ]:
#applying gaussian blur to smoothen out the image edges
filter2 = cv2.GaussianBlur(filter1,(7,7),0)
cv2_imshow(filter2)

In [ ]:
#applying non-localized means for final Denoising of the image
dst = cv2.fastNlMeansDenoising(filter2,None,11,3,11)
cv2_imshow(dst)

In [ ]:
#converting the image to binarized form using adaptive thresholding
th1 = cv2.adaptiveThreshold(dst,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,19,2)

In [ ]:
cv2.imwrite('ImagePreProcessingFinal2.jpg', th1)

In [ ]:
from google.colab.patches import cv2_imshow


In [ ]:
cv2_imshow(th1)

In [ ]:
cv2_imshow(image)